In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scipy.conftest import num_parallel_threads

import my_afml_fncs as my_afml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
filename_end = "2019_2024"

dollar_bars_df = pd.read_csv(f'tick_data/bars/dollar_bars_barchart_{filename_end}.csv')
dollar_bars_df['bar_number'] = range(len(dollar_bars_df))
dollar_bars_df['Date'] = pd.to_datetime(dollar_bars_df['Date'])
dollar_bars_df.set_index('Date', inplace=True)
dollar_bars_df.sort_index(inplace=True)

In [3]:
t1 = pd.Series([2, 3, 5], index=[0, 2, 4])  # t0,t1 for each feature obs
barIx = range(t1.max() + 1)  # index of bars
indM = my_afml.getIndMatrix(barIx, t1)
phi = np.random.choice(indM.columns, size=indM.shape[1])
print(phi)
print('Standard uniqueness:', my_afml.getAvgUniqueness(indM[phi]).mean())
phi = my_afml.seqBootstrap(indM)
print(phi)
print('Sequential uniqueness:',my_afml.getAvgUniqueness(indM[phi]).mean())


[0 1 1]
Standard uniqueness: 0.5370370370370371


Sequential Bootstrap: 100%|██████████| 3/3 [00:00<00:00, 285.73it/s]

[0, 2, 0]
Sequential uniqueness: 0.6666666666666666


In [4]:
dollar_vol=my_afml.getDailyVol(dollar_bars_df.Close)

In [5]:
# 3.1(a) apply CUSUM filet usings daily vols
dollar_es_cusum_events=my_afml.getTEvents(dollar_bars_df.Close,dollar_vol.mean())
print('Dollar bar CUSUM events:',dollar_es_cusum_events.shape)

Dollar bar CUSUM events: (48245,)


In [6]:
# Start of 3.4
# Define moving average windows
short_window = 20  # e.g., 50 periods
long_window = 50  # e.g., 200 periods

In [7]:
# create events where the moving averages cross over
dollar_events = my_afml.movingAverageCrossover(dollar_bars_df.Close, short_window, long_window)
# try this with a crossover sign change
dollar_events['signal_switch'] = dollar_events.signal.diff() / 2
# drop rows with any NaN values
dollar_events = dollar_events.dropna()
# filter rows where signal_switch is not zero
dollar_events = dollar_events[dollar_events['signal_switch'] != 0]

In [8]:
dollar_t1=my_afml.getVb(dollar_bars_df.Close,dollar_events.index)

In [9]:
indM=my_afml.getIndMatrix(dollar_events.index,dollar_t1)

In [10]:
# phi = np.random.choice(indM.columns, size=indM.shape[1])
# print(phi)
# print('Standard uniqueness:', my_afml.getAvgUniqueness(indM[phi]).mean())
# phi = my_afml.seqBootstrap(indM)
# print(phi)

In [11]:
# out['w']=mpPandasObj(mpSampleW,('molecule',events.index),numThreads, \
# t1=events['t1'],numCoEvents=numCoEvents,close=close)
# out['w']*=out.shape[0]/out['w'].sum()

In [12]:
dollar_bars_2024 = dollar_bars_df[dollar_bars_df.index.year == 2024]
dollar_events_2024 = dollar_events[dollar_events.index.year == 2024]

In [13]:
dollar_t1_2024=my_afml.getVb(dollar_bars_2024.Close,dollar_events_2024.index)